# Evaluating truth vs sourmash gather



In [3]:
import csv
import sys
sys.path.insert(0, '../2018-ncbi-lineages/')
import ncbi_taxdump_utils
import os

in ../2018-ncbi-lineages

    mkdir genbank

    cd genbank/

    curl -L -O ftp://ftp.ncbi.nih.gov/pub/taxonomy//taxdmp.zip
    unzip taxdmp.zip nodes.dmp names.dmp
    rm taxdmp.zip


In [7]:
taxfoo = ncbi_taxdump_utils.NCBI_TaxonomyFoo()

taxfoo.load_nodes_dmp('../2018-ncbi-lineages/genbank/nodes.dmp')
taxfoo.load_names_dmp('../2018-ncbi-lineages/genbank/names.dmp')

want_taxonomy = ['superkingdom', 'phylum', 'order', 'class', 'family', 'genus', 'species']

In [18]:
from sourmash.lca import lca_utils, command_index

def format_lineage(lineage_tup):
    return ";".join(lca_utils.zip_lineage(lineage_tup))

# Sourmash gather: mapping from accession -> lineages
This is important for dealing with the CSV output of sourmash gather; sourmash lca gather comes with the taxonomy already in the CSV output.

In [19]:
acc_to_lineage, num_rows = command_index.load_taxonomy_assignments('gather-lineages.csv', start_column=3)
print('loaded {} rows'.format(num_rows))

loaded 823 rows


examining spreadsheet headers...
** assuming column 'accession' is identifiers in spreadsheet


In [20]:
def get_lineage_by_acc(acc):
    acc = acc.split(' ')[0].split('.')[0]
    return acc_to_lineage.get(acc, None)

# example for 'get_lineage_by_acc'
#lineage = get_lineage_by_acc('NC_000917')
#print(lineage)
#print(format_lineage(lineage))

In [24]:
def load_sourmash_csv(filename):
    with open(filename, 'rt') as fp:
        r = csv.DictReader(fp)
        rows = list(r)
    return rows

load_sourmash_csv('output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')[:1]

[OrderedDict([('intersect_bp', '330000'),
              ('f_orig_query', '0.008465879938429965'),
              ('f_match', '0.03466386554621849'),
              ('f_unique_to_query', '0.008465879938429965'),
              ('f_unique_weighted', '0.008465879938429965'),
              ('average_abund', '1.0'),
              ('name',
               'NC_013132.1 Chitinophaga pinensis DSM 2588, complete genome'),
              ('filename', 'inputs/databases/genbank-k51.sbt.json'),
              ('md5', 'd06a405d39c8ebed579540db2994afe8')])]

In [189]:
notfound = set()
def make_gather_lineages(filename):
    rows = load_sourmash_csv(filename)
    rows2 = []
    for d in rows:
        name = d['name']
        lineage = get_lineage_by_acc(name)
        if lineage is None:
            print('ZZZ found no lineage for {}'.format(name))
            notfound.add(name)
            continue
        lineage = [x for x in lineage if x.rank != "strain"]
        lineage = tuple(lineage)
        d2 = dict(d)
        d2['lineage'] = lineage
        rows2.append(d2)
        
    return rows2

make_gather_lineages('output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')[:1]

[{'average_abund': '1.0',
  'f_match': '0.03466386554621849',
  'f_orig_query': '0.008465879938429965',
  'f_unique_to_query': '0.008465879938429965',
  'f_unique_weighted': '0.008465879938429965',
  'filename': 'inputs/databases/genbank-k51.sbt.json',
  'intersect_bp': '330000',
  'lineage': (LineagePair(rank='superkingdom', name='Bacteria'),
   LineagePair(rank='phylum', name='Bacteroidetes'),
   LineagePair(rank='class', name='Chitinophagia'),
   LineagePair(rank='order', name='Chitinophagales'),
   LineagePair(rank='family', name='Chitinophagaceae'),
   LineagePair(rank='genus', name='Chitinophaga'),
   LineagePair(rank='species', name='Chitinophaga pinensis')),
  'md5': 'd06a405d39c8ebed579540db2994afe8',
  'name': 'NC_013132.1 Chitinophaga pinensis DSM 2588, complete genome'}]

## Truth files: loading in & fleshing out NCBI taxonomy
From McIntyre et al. 2018 https://ftp-private.ncbi.nlm.nih.gov/nist-immsa/IMMSA/

In [171]:
def load_truth_file(filename):
    with open(filename, 'rt') as fp:
        lines = fp.readlines()
        
    lines = [ x.strip() for x in lines ]
    lines = [ x.split('\t') for x in lines ]
    
    rows = []
    for x in lines:
        taxid, a, b, rank, name = x
        taxid = int(taxid)
        rows.append((taxid, a, b, rank, name))
    return rows
    
print(load_truth_file('truth_sets/species/Huttenhower_HC1_TRUTH.txt')[:5])

[(84980, '1.00000', '0.01000', 'species', 'Desulfotalea psychrophila'), (1428, '1.00000', '0.01000', 'species', 'Bacillus thuringiensis'), (1496, '1.00000', '0.01000', 'species', 'Peptoclostridium difficile'), (1525, '1.00000', '0.01000', 'species', 'Moorella thermoacetica'), (1423, '1.00000', '0.01000', 'species', 'Bacillus subtilis')]


In [101]:
def make_lineage_from_taxid(taxid):
    lineage_d = taxfoo.get_lineage_as_dict(taxid, want_taxonomy)
    
    lineage = []
    for rank in lca_utils.taxlist():
        if rank == 'strain': continue 
        name = lineage_d.get(rank, 'unassigned')
        lineage_pair = lca_utils.LineagePair(rank, name)
        lineage.append(lineage_pair)
    return tuple(lineage)

make_lineage_from_taxid(84980)

(LineagePair(rank='superkingdom', name='Bacteria'),
 LineagePair(rank='phylum', name='Proteobacteria'),
 LineagePair(rank='class', name='Deltaproteobacteria'),
 LineagePair(rank='order', name='Desulfobacterales'),
 LineagePair(rank='family', name='Desulfobulbaceae'),
 LineagePair(rank='genus', name='Desulfotalea'),
 LineagePair(rank='species', name='Desulfotalea psychrophila'))

In [30]:
def make_truth_lineages(filename):
    rows = load_truth_file(filename)
    
    rows2 = []
    for (taxid, a, b, rank, name) in rows:
        lineage = make_lineage_from_taxid(taxid)
        rows2.append((taxid, a, b, rank, name, lineage))
        for lintup in lineage:
            if lintup.rank == rank:
                if lintup.name != name:
                    print('DISAGREE: ncbi={}, truthfile={}'.format(lintup.name, name))
    return rows2

truth_lineages = make_truth_lineages('truth_sets/species/Huttenhower_HC1_TRUTH.txt')
print(truth_lineages[:1])

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
[(84980, '1.00000', '0.01000', 'species', 'Desulfotalea psychrophila', (LineagePair(rank='superkingdom', name='Bacteria'), LineagePair(rank='phylum', name='Proteobacteria'), LineagePair(rank='class', name='Deltaproteobacteria'), LineagePair(rank='order', name='Desulfobacterales'), LineagePair(rank='family', name='Desulfobulbaceae'), LineagePair(rank='genus', name='Desulfotalea'), LineagePair(rank='species', name='Desulfotalea psychrophila'), LineagePair(rank='strain', name='unassigned')))]


## Sourmash gather: mapping from accession -> lineages

This is important for dealing with the CSV output of `sourmash gather`; `sourmash lca gather`
comes with the taxonomy already in the CSV output.

In [31]:
def make_gather_lineages(filename):
    rows = load_sourmash_csv(filename)
    rows2 = []
    for d in rows:
        d2 = dict(d)
        
        lineage = []
        for rank in lca_utils.taxlist():
            if rank in d2:
                name = d2.get(rank)
                del d2[rank]
                lineage.append((rank, name))
                
        lineage = [ lca_utils.LineagePair(r, n) for (r, n) in lineage ]
        d2['lineage'] = tuple(lineage)
        rows2.append(d2)
        
    return rows2

make_gather_lineages('output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')[:1]

[{'average_abund': '1.0',
  'f_match': '0.03466386554621849',
  'f_orig_query': '0.008465879938429965',
  'f_unique_to_query': '0.008465879938429965',
  'f_unique_weighted': '0.008465879938429965',
  'filename': 'inputs/databases/genbank-k51.sbt.json',
  'intersect_bp': '330000',
  'lineage': (),
  'md5': 'd06a405d39c8ebed579540db2994afe8',
  'name': 'NC_013132.1 Chitinophaga pinensis DSM 2588, complete genome'}]

## Load LCA gather csv files

In [89]:
def make_lca_gather_lineages(filename):
    rows = load_sourmash_csv(filename)
    rows2 = []
    for d in rows:
        d2 = dict(d)
        
        lineage = []
        for rank in lca_utils.taxlist():
            if rank in d2:
                name = d2.get(rank)
                del d2[rank]
                lineage.append((rank, name))
                
        lineage = [ lca_utils.LineagePair(r, n) for (r, n) in lineage ]
        d2['lineage'] = tuple(lineage)
        rows2.append(d2)
        
    return rows2

make_lca_gather_lineages('output/Huttenhower_HC1.fasta.gz.scaled10k.k51.lca.gather.matches.csv')[:1]

[{'average_abund': '1.0',
  'f_unique_to_query': '0.013083632632119035',
  'f_unique_weighted': '0.013083632632119035',
  'intersect_bp': '510000',
  'lineage': (LineagePair(rank='superkingdom', name='Bacteria'),
   LineagePair(rank='phylum', name='Firmicutes'),
   LineagePair(rank='class', name='Bacilli'),
   LineagePair(rank='order', name='Bacillales'),
   LineagePair(rank='family', name='Bacillaceae'),
   LineagePair(rank='genus', name='Bacillus'),
   LineagePair(rank='species', name='Bacillus subtilis'))}]

In [33]:
ls

README.md                     get-names-from-gather-csv.py*
Untitled.ipynb                output/
gather-accessions.txt         parse.ipynb
gather-accessions.txt.taxid   truth_sets/


In [34]:
ls output/Huttenhower_HC1*

output/Huttenhower_HC1.fasta.gz.scaled10k.k51.lca.gather.matches.csv
output/Huttenhower_HC1.fasta.gz_contigs.scaled10k.k51.lca.gather.matches.csv
output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv
output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv


In [35]:
ls truth_sets/species/Huttenhower_HC1*

truth_sets/species/Huttenhower_HC1_TRUTH.txt


In [36]:
def compare_gather_to_truth(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_gather_lineages(gather_csv)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    print(len(truth_lineages.intersection(gather_lineages)))
    print(len(truth_lineages.union(gather_lineages)))
    
    print(len(truth_lineages))
    
    print('** in gather but not truth:')
    for diff in gather_lineages - truth_lineages:
        print('\t', format_lineage(diff))
    
    print('\n** in truth but not gather:')
    for diff in truth_lineages - gather_lineages:
        print('\t', format_lineage(diff))
    
compare_gather_to_truth('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')


DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
0
101
100
** in gather but not truth:
	 ;;;;;;;

** in truth but not gather:
	 Bacteria;Firmicutes;Bacilli;Bacillales;Bacillaceae;Bacillus;Bacillus atrophaeus;unassigned
	 Bacteria;Proteobacteria;Deltaproteobacteria;Desulfobacterales;Desulfobulbaceae;Desulfotalea;Desulfotalea psychrophila;unassigned
	 Bacteria;Proteobacteria;Betaproteobacteria;Burkholderiales;Burkholderiaceae;Ralstonia;Ralstonia pickettii;unassigned
	 Bacteria;Proteobacteria;Epsilonproteobacteria;Campylobacterales;Helicobacteraceae;Helicobacter;Helicobacter acinonychis;unassigned
	 Bacteria;Firmicutes;Bacilli;Lactobacillales;Lactobacillaceae;Lactobacillus;Lactobacillus helveticus;unassigned
	 Bacteria;Proteobacteria;Epsilonproteobacteria;Campylobacterales;Helicobacteraceae;Helicobacter;Helicobacter mustelae;unassigned
	 Bacteria;Proteobacteria;Betaproteobacteria

In [18]:
def compare_lca_gather_to_truth(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_lca_gather_lineages(gather_csv)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    print(len(truth_lineages.intersection(gather_lineages)))
    print(len(truth_lineages.union(gather_lineages)))
    
    print(len(truth_lineages))
    
    print('** in gather but not truth:')
    for diff in gather_lineages - truth_lineages:
        print('\t', format_lineage(diff))
    
    print('\n** in truth but not gather:')
    for diff in truth_lineages - gather_lineages:
        print('\t', format_lineage(diff))
    
compare_lca_gather_to_truth('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv')


DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
97
106
100
** in gather but not truth:
	 Bacteria;Firmicutes;Bacilli;Lactobacillales;Lactobacillaceae;Lactobacillus;Lactobacillus gallinarum
	 Bacteria;Proteobacteria;Gammaproteobacteria;Vibrionales;Vibrionaceae;Vibrio;Vibrio hyugaensis
	 Bacteria;Proteobacteria;Epsilonproteobacteria;Campylobacterales;Helicobacteraceae;Helicobacter;Helicobacter pylori
	 Bacteria;Proteobacteria;Betaproteobacteria;Rhodocyclales;Rhodocyclaceae;Dechloromonas;Dechloromonas aromatica
	 Bacteria;Proteobacteria;Gammaproteobacteria;Enterobacterales;Hafniaceae;Edwardsiella;Edwardsiella piscicida
	 Bacteria;Firmicutes;Bacilli;Bacillales;Bacillaceae;Bacillus;Bacillus sp. GZT

** in truth but not gather:
	 Bacteria;Firmicutes;Bacilli;Bacillales;Bacillaceae;Bacillus;Bacillus atrophaeus
	 Bacteria;Proteobacteria;Betaproteobacteria;Rhodocyclales;Azonexaceae;Dec

In [37]:
def compare_lca_gather_to_gather(lca_gather_csv, gather_csv):
    reg_gather = make_gather_lineages(gather_csv)
    lca_gather = make_lca_gather_lineages(lca_gather_csv)
    
    reg_gather_lineages = set([ row['lineage'] for row in reg_gather ])
    lca_gather_lineages = set([ row['lineage'] for row in lca_gather ])
    
    print(len(lca_gather_lineages.intersection(reg_gather_lineages)))
    print(len(lca_gather_lineages.union(reg_gather_lineages)))
        
    print('** in lca_gather but not reg gather:')
    for diff in lca_gather_lineages - reg_gather_lineages:
        print('\t', format_lineage(diff))
    
    print('\n** in gather but not lca gather:')
    for diff in reg_gather_lineages - lca_gather_lineages:
        print('\t', format_lineage(diff))
    
compare_lca_gather_to_gather('output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv',
                            'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')


0
104
** in lca_gather but not reg gather:
	 Bacteria;Proteobacteria;Epsilonproteobacteria;Campylobacterales;Helicobacteraceae;Helicobacter;Helicobacter hepaticus;
	 Bacteria;Deinococcus-Thermus;Deinococci;Thermales;Thermaceae;Thermus;Thermus thermophilus;
	 Bacteria;Proteobacteria;Alphaproteobacteria;Rickettsiales;Rickettsiaceae;Rickettsia;Rickettsia akari;
	 Bacteria;Proteobacteria;Gammaproteobacteria;Alteromonadales;Shewanellaceae;Shewanella;Shewanella sediminis;
	 Bacteria;Firmicutes;Bacilli;Bacillales;Staphylococcaceae;Macrococcus;Macrococcus caseolyticus;
	 Bacteria;Firmicutes;Bacilli;Bacillales;Bacillaceae;Bacillus;Bacillus cereus;
	 Bacteria;Firmicutes;Bacilli;Lactobacillales;Streptococcaceae;Streptococcus;Streptococcus thermophilus;
	 Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanothermaceae;Methanothermus;Methanothermus fervidus;
	 Bacteria;Chlorobi;Chlorobia;Chlorobiales;Chlorobiaceae;Pelodictyon;Pelodictyon luteolum;
	 Archaea;Crenarchaeota;Thermoprotei;The

In [38]:
all_truth = """ABRF_MGRG_10ng_TRUTH.txt
ABRF_MGRG_1ng_TRUTH.txt
ABRF_MGRG_5ng_TRUTH.txt
ABRF_MGRG_Half_TRUTH.txt
ABRF_MGRG_Normal_TRUTH.txt
ABRF_MGRG_classIplus_TRUTH.txt
BMI_bmi_reads_TRUTH.txt
BioPool_BioPool_TRUTH.txt
BioPool_BioPool_abundances.txt
Carma_eval_carma_TRUTH.txt
HMP_even_454_SRR072233_TRUTH.txt
HMP_even_illum_SRR172902_TRUTH.txt
Huttenhower_HC-LC_abundances.txt
Huttenhower_HC1_TRUTH.txt
Huttenhower_HC2_TRUTH.txt
Huttenhower_LC1_TRUTH.txt
Huttenhower_LC2_TRUTH.txt
Huttenhower_LC3_TRUTH.txt
Huttenhower_LC4_TRUTH.txt
Huttenhower_LC5_TRUTH.txt
Huttenhower_LC6_TRUTH.txt
Huttenhower_LC7_TRUTH.txt
Huttenhower_LC8_TRUTH.txt
JGI_SRR033547_TRUTH.txt
JGI_SRR033548_TRUTH.txt
JGI_SRR033549_TRUTH.txt
Mavromatis_simHC_TRUTH.txt
Mavromatis_simLC_TRUTH.txt
Mavromatis_simMC_TRUTH.txt
Raiphy_eval_RAIphy_TRUTH.txt
UnAmbiguouslyMapped_ds.7_TRUTH.txt
UnAmbiguouslyMapped_ds.buccal_TRUTH.txt
UnAmbiguouslyMapped_ds.cityparks_TRUTH.txt
UnAmbiguouslyMapped_ds.gut_TRUTH.txt
UnAmbiguouslyMapped_ds.hous1_TRUTH.txt
UnAmbiguouslyMapped_ds.hous2_TRUTH.txt
UnAmbiguouslyMapped_ds.nycsm_TRUTH.txt
UnAmbiguouslyMapped_ds.soil_TRUTH.txt"""

all_truth = all_truth.splitlines()
all_truth = [ x[:-10] for x in all_truth ]
print(all_truth)

['ABRF_MGRG_10ng', 'ABRF_MGRG_1ng', 'ABRF_MGRG_5ng', 'ABRF_MGRG_Half', 'ABRF_MGRG_Normal', 'ABRF_MGRG_classIplus', 'BMI_bmi_reads', 'BioPool_BioPool', 'BioPool_BioPool_abun', 'Carma_eval_carma', 'HMP_even_454_SRR072233', 'HMP_even_illum_SRR172902', 'Huttenhower_HC-LC_abun', 'Huttenhower_HC1', 'Huttenhower_HC2', 'Huttenhower_LC1', 'Huttenhower_LC2', 'Huttenhower_LC3', 'Huttenhower_LC4', 'Huttenhower_LC5', 'Huttenhower_LC6', 'Huttenhower_LC7', 'Huttenhower_LC8', 'JGI_SRR033547', 'JGI_SRR033548', 'JGI_SRR033549', 'Mavromatis_simHC', 'Mavromatis_simLC', 'Mavromatis_simMC', 'Raiphy_eval_RAIphy', 'UnAmbiguouslyMapped_ds.7', 'UnAmbiguouslyMapped_ds.buccal', 'UnAmbiguouslyMapped_ds.cityparks', 'UnAmbiguouslyMapped_ds.gut', 'UnAmbiguouslyMapped_ds.hous1', 'UnAmbiguouslyMapped_ds.hous2', 'UnAmbiguouslyMapped_ds.nycsm', 'UnAmbiguouslyMapped_ds.soil']


In [39]:
triples = []
for prefix in all_truth:
    truth_file = 'truth_sets/species/{}_TRUTH.txt'.format(prefix)
    lca_gather_file = 'output/{}.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv'.format(prefix)
    gather_file = 'output/{}.fasta.gz_reads.scaled10k.k51.gather.matches.csv'.format(prefix)
    
    if os.path.exists(truth_file) and os.path.exists(lca_gather_file) and os.path.exists(gather_file):
        triples.append((truth_file, lca_gather_file, gather_file))

In [40]:
#make dictionary 
triple_dict = {}
for x in triples: 
    temp_dict = {}
    temp_dict['lca_gather'] = x[1]
    temp_dict['gather'] = x[2]
    triple_dict[x[0]] = temp_dict


#for key in triple_dict.keys(): print (key)

for k,v in triple_dict.items(): 
    print (v)
    print (k)

{'lca_gather': 'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv', 'gather': 'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv'}
truth_sets/species/Huttenhower_HC1_TRUTH.txt
{'lca_gather': 'output/Huttenhower_HC2.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv', 'gather': 'output/Huttenhower_HC2.fasta.gz_reads.scaled10k.k51.gather.matches.csv'}
truth_sets/species/Huttenhower_HC2_TRUTH.txt
{'lca_gather': 'output/Huttenhower_LC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv', 'gather': 'output/Huttenhower_LC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv'}
truth_sets/species/Huttenhower_LC1_TRUTH.txt
{'lca_gather': 'output/Huttenhower_LC2.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv', 'gather': 'output/Huttenhower_LC2.fasta.gz_reads.scaled10k.k51.gather.matches.csv'}
truth_sets/species/Huttenhower_LC2_TRUTH.txt
{'lca_gather': 'output/Huttenhower_LC3.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv', 'gather': 'output/H

Goal: Compare lineages corresponding to gather output to truth set 


In [175]:
#Calculate false positives 
def compare_lca_gather_to_truth_fp(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_lca_gather_lineages(gather_csv)
    #print(gather)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    #print(len(truth_lineages))
    #print(len(gather_lineages))
    #print(len(gather_lineages - truth_lineages))
    return len(gather_lineages - truth_lineages)
    
compare_lca_gather_to_truth_fp('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv')

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae


6

In [176]:
#Calculate false positives 
def compare_gather_to_truth_fp(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_gather_lineages(gather_csv)
    #print(gather)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    #print(len(truth_lineages))
    #print(len(gather_lineages))
    #print(len(gather_lineages - truth_lineages))
    return len(gather_lineages - truth_lineages)
    
compare_gather_to_truth_fp('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae


2

In [177]:
#Calculate true positives 
#True positives = intersection  of truth set and experimental set 

def compare_lca_gather_to_truth_tp(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_lca_gather_lineages(gather_csv)
    #print(gather)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    #print(len(truth_lineages))
    #print(len(gather_lineages))
    #print(len(truth_lineages.intersection(gather_lineages)))
    #pprint.pprint(truth_lineages - gather_lineages)
    return len(truth_lineages.intersection(gather_lineages))



compare_lca_gather_to_truth_tp('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv')

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae


97

In [178]:
#Calculate true positives 
#True positives = intersection  of truth set and experimental set 

def compare_gather_to_truth_tp(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_gather_lineages(gather_csv)
    #print(gather)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    #print(len(truth_lineages))
    #print(len(gather_lineages))
    #print(len(truth_lineages.intersection(gather_lineages)))
    #pprint.pprint(truth_lineages - gather_lineages)
    return len(truth_lineages.intersection(gather_lineages))

compare_gather_to_truth_tp('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae


92

In [179]:
#Calculate false negatives
#Compare the lists, calculate union, and subtract length of truth_set from length of union 
import pprint 

def compare_lca_gather_to_truth_fn(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_lca_gather_lineages(gather_csv)
    
    #pprint.pprint(truth)
    #pprint.pprint(gather)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    #pprint.pprint(len(truth_lineages - gather_lineages))
    
    return len(truth_lineages - gather_lineages)

false_negatives = compare_lca_gather_to_truth_fn('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv')


DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae


In [180]:
#Calculate false negatives
#Compare the lists, calculate union, and subtract length of truth_set from length of union 
import pprint 

def compare_gather_to_truth_fn(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_gather_lineages(gather_csv)
    
    #pprint.pprint(truth)
    #pprint.pprint(gather)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    #pprint.pprint(len(truth_lineages - gather_lineages))
    
    return len(truth_lineages - gather_lineages)

false_negatives = compare_gather_to_truth_fn('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')



DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae


In [181]:
len(open('output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv').readlines())

104

In [182]:
false_negatives

8

In [183]:
len(open("truth_sets/species/Huttenhower_HC1_TRUTH.txt").readlines())

100

In [184]:
def summarize_lca_gather(truth, lca_gather):
    
    tp = compare_lca_gather_to_truth_tp(truth, lca_gather)
    
    fp = compare_lca_gather_to_truth_fp(truth, lca_gather)
    
    fn = compare_lca_gather_to_truth_fn(truth, lca_gather)
    
    #Precision = true_positve/true positve + false postive 

    precision = tp/(tp + fp)

    #Recall = true_positve/true positve + false negative 

    recall = tp/(tp + fn)

    #print(precision, recall)
    return precision, recall
p, r = summarize('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                            'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.lca.gather.matches.csv')
print(p, r)

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
0.941747572815534 0.97


In [190]:
def summarize_gather(truth, lca_gather):
    
    tp = compare_gather_to_truth_tp(truth, gather)
    
    fp = compare_gather_to_truth_fp(truth, gather)
    
    fn = compare_gather_to_truth_fn(truth, gather)
    
    #Precision = true_positve/true positve + false postive 

    precision = tp/(tp + fp)

    #Recall = true_positve/true positve + false negative 

    recall = tp/(tp + fn)

    #print(precision, recall)
    return precision, recall
p, r = summarize_gather('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                            'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')
print(p, r)

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
0.1111111111111111 0.01


In [191]:

results = []

for truth, lca_gather, gather in triples: 
    p, r = summarize_lca_gather(truth, lca_gather)
    print(p, r)
    
    p2, r2 = summarize_gather(truth, gather)
    results.append((truth, p, r, p2, r2))
    

DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
0.941747572815534 0.97
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
0.913461538461538

In [187]:
# Precision and recall for lca gather 
import pandas as pd 
df = pd.DataFrame(results)
df

,0,1,2,3,4
0,truth_sets/species/Huttenhower_HC1_TRUTH.txt,0.941748,0.97,0.978723,0.92
1,truth_sets/species/Huttenhower_HC2_TRUTH.txt,0.913462,0.95,0.960000,0.48
2,truth_sets/species/Huttenhower_LC1_TRUTH.txt,0.700000,0.56,0.818182,0.36
3,truth_sets/species/Huttenhower_LC2_TRUTH.txt,0.916667,0.88,1.000000,0.64
4,truth_sets/species/Huttenhower_LC3_TRUTH.txt,0.818182,0.72,1.000000,0.40
5,truth_sets/species/Huttenhower_LC4_TRUTH.txt,0.800000,0.48,1.000000,0.20
6,truth_sets/species/Huttenhower_LC5_TRUTH.txt,0.529412,0.36,0.700000,0.28
7,truth_sets/species/Huttenhower_LC6_TRUTH.txt,1.000000,0.68,1.000000,0.40
8,truth_sets/species/Huttenhower_LC7_TRUTH.txt,0.866667,0.52,1.000000,0.24
9,truth_sets/species/Huttenhower_LC8_TRUTH.txt,0.850000,0.68,1.000000,0.36


In [196]:
print("\n".join([x.split()[0] for x in notfound]))

JYCX01000001.1
MIIR01000001.1
CP002530.1
AP008971.1
FN543502.1
CP001712.1
JANW01000001.1
AM743169.1
KL411729.1
CP001738.1
JQEP01000001.1
CP002394.1
CP000139.1
CP001047.1
FN692037.1
CR954246.1
CR931997.1
CP001840.1
CP009601.1
JGYA01000001.1
CP000822.1
CP000284.1
CP000034.1
KQ758435.1
BX571963.1
AFMU01000060.1
CP001189.1
LITS01000001.1
CQJG01000001.1
CP001011.1
CP002343.1
CP006471.1
DS989821.1
FR687359.1
BABR01000546.1
LANS01000001.1
CP000812.1
CP002008.1
CP001279.1
KB932555.1
CP012729.1


In [122]:
# Calculate false positives 
def compare_gather_to_truth(truth_file, gather_csv):
    truth = make_truth_lineages(truth_file)
    gather = make_gather_lineages(gather_csv)
    #print(gather)
    
    truth_lineages = set([ t[5] for t in truth ])
    gather_lineages = set([ row['lineage'] for row in gather ])
    
    print(len(truth_lineages))
    print(len(gather_lineages))
    print(len(gather_lineages - truth_lineages))
    return len(gather_lineages - truth_lineages)

false_positives = compare_gather_to_truth('truth_sets/species/Huttenhower_HC1_TRUTH.txt',
                        'output/Huttenhower_HC1.fasta.gz_reads.scaled10k.k51.gather.matches.csv')


DISAGREE: ncbi=Clostridioides difficile, truthfile=Peptoclostridium difficile
DISAGREE: ncbi=Sediminispirochaeta smaragdinae, truthfile=Spirochaeta smaragdinae
100
94
2


In [123]:
false_positives